# Credential Access - Browser Credential Dumping

#### Author: Rifqi Ardia Ramadhan

There is a useful feature for browser that let you save credentials so you won't have to type those safe-but-too-long-too-remember again. You got another business to do, right? But this convenience comes with a risk. This credentials are most likely saved locally to storage. When an adversary already have an access into your computer without you even realize it, they can use it to get access to read credentials in each of your browsers. In this article, I want to show you how to detect if an application (other than the browser itself of course) is reading data from credential storages. One of the famous credential dumper is LaZagne Project.

## Use Case
### Objective
Identify if there is any process other than browsers that accessed browser-saved credentials.

### Log Requirements
Microsoft Windows Security Auditing

### Characteristics
| No | Characteristic | Note | Why|
|----|----------------|------|---|
|  1 | A 4663 EventID is logged| | This event is too vague, you need another characteristics to support the investigation |
|2   | The file accessed is browser's password database || |
|3   | The program that accessed the browser is not related to any browser | | Note every browser's default directory, and see if it's related|
|4   | AccessList is 4416 || This access code means that the program is simply reading the data, not writing it|

### References
https://attack.mitre.org/techniques/T1555/003/

https://github.com/AlessandroZ/LaZagne

https://social.technet.microsoft.com/Forums/windows/en-US/541bad5d-19eb-4de5-8ef7-1b144f0b6113/translate-xxxx-values-in-events?forum=w7itprosecurity

### Sample
Dataset acquired from [EVTX-ATTACK-SAMPLES](https://github.com/sbousseaden/EVTX-ATTACK-SAMPLES)

In [4]:
import evtx
import xml.etree.ElementTree as ET
import json

evtx_file = "logs/Credential Access/CA_chrome_firefox_opera_4663.evtx"  # change the source file to the one you have

parser = evtx.PyEvtxParser(evtx_file)
parse_json = list(parser.records())

# all of the above is just to parse the EVTX file as Dictionary

# This is used to get the object that was accessed
browsers = ["Google\\", "Firefox\\", "Opera Software\\"]    # just add it with whatever directory your browser you use, Chromium, Brave or something

for i in parse_json:
    if "<EventID>4663</EventID>" in i['data'] and any(browser in i['data'] for browser in browsers) and "\"AccessList\">%%4416" in i['data']:
        print(i['data'].replace("&quot;", "\""))

<?xml version="1.0" encoding="utf-8"?>
<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">
  <System>
    <Provider Name="Microsoft-Windows-Security-Auditing" Guid="54849625-5478-4994-A5BA-3E3B0328C30D">
    </Provider>
    <EventID>4663</EventID>
    <Version>0</Version>
    <Level>0</Level>
    <Task>12800</Task>
    <Opcode>0</Opcode>
    <Keywords>0x8020000000000000</Keywords>
    <TimeCreated SystemTime="2019-04-27 19:33:50.134293 UTC">
    </TimeCreated>
    <EventRecordID>4991</EventRecordID>
    <Correlation>
    </Correlation>
    <Execution ProcessID="4" ThreadID="44">
    </Execution>
    <Channel>Security</Channel>
    <Computer>IEWIN7</Computer>
    <Security>
    </Security>
  </System>
  <EventData>
    <Data Name="SubjectUserSid">S-1-5-21-3583694148-1414552638-2922671848-1000</Data>
    <Data Name="SubjectUserName">IEUser</Data>
    <Data Name="SubjectDomainName">IEWIN7</Data>
    <Data Name="SubjectLogonId">0xffa8</Data>
    <Data Name="ObjectServer">